## Healthcare Agent Service Integration

Use [Healthcare agent service](https://learn.microsoft.com/en-us/azure/health-bot/overview) Agent's.
This notebook assumes the use of the contents of `healthcare_agents.yaml` file in place of `agents.yaml`.
Single agent execution is covered in this notebook.

In [ ]:
import logging
import os
import sys
sys.path.append('../../src')

from dotenv import load_dotenv
load_dotenv("../../src/.env")

from semantic_kernel.contents.utils.author_role import AuthorRole
from azure.storage.blob.aio import BlobServiceClient

from config import setup_logging
from data_models.data_access import DataAccess
from data_models.chat_context import ChatContext
from healthcare_agents import HealthcareAgent

setup_logging(log_level=logging.WARNING)
logging.getLogger("healthcare_agents").setLevel(logging.DEBUG)

logger = logging.getLogger(__name__)


In [ ]:
# Initialize all agent engagements
engagements = {
    "Orchestrator": {
        "agent_name": "Orchestrator",
        "prompts": [
            "can you help me prepare for a tumor board meeting later today",
            "actually, do radiology last, before the summary",
            "proceed with patient ID patient_4",
        ]
    },
    "PatientStatus": {
        "agent_name": "PatientStatus",
        "prompts": [
            "can you give me the status of patient ID patient_4"
        ]
    },
    "ClinicalTrials": {
        "agent_name": "ClinicalTrials",
        "prompts": [
            "can you give me clinical trials suitable for patient ID patient_4"
        ]
    },
    "ClinicalGuidelines": {
        "agent_name": "ClinicalGuidelines",
        "prompts": [
            """ <insert patient details here>
                please give me relevant treatment guidelines for this patient.
            """
        ]
    },
    "Radiology": {
        "agent_name": "Radiology",
        "prompts": [
            "analyze the latest CT scan for patient ID patient_4"
        ]
    },
}

In [ ]:
# Pick an agent to engage with
# engagement = "Orchestrator"
# engagement = "PatientStatus"
# engagement = "ClinicalTrials"
engagement = "ClinicalGuidelines"
# engagement = "Radiology"

engage_with_agent = engagements.get(engagement)

In [ ]:
from app import create_app_context


app_ctx = create_app_context()
agent_name = engage_with_agent["agent_name"]
chat_ctx = ChatContext(f"{agent_name}-test-chat")
healthcare_agent = HealthcareAgent(name=agent_name,
                                   chat_ctx=chat_ctx,
                                   app_ctx=app_ctx,)

In [ ]:
# Invoke Agent
prompts = engage_with_agent.get("prompts", [])

async def chat(message: str):
    logger.info(f"Chatting with [{healthcare_agent.name} agent], sending message: [{message}]")
    async for msg in healthcare_agent.invoke(message=message):
        if msg.role != AuthorRole.USER:
            print(f"{msg.role} [{msg.name}]: {msg.content}")

for prompt in prompts:
    await chat(message=prompt)
